In [ ]:
# Sample code for social learning PSO, using DEAP library
import operator
import random
import numpy as np
import math
from math import sqrt
import torch.nn as nn
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt
!pip install deap
from deap import base
from deap import benchmarks
from deap import creator
from deap import tools

# Load Dataset
batch_size = 10000

transform   = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) # Function to transform the dataset to the specified range

trainset      = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader   = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
testset       = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader    = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)
classes       = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

device        = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print("Device, ", device)

totalImages   = len(trainloader.dataset)
miniAmount    = 1000
numOfLoaders  = totalImages // miniAmount

print("Number of loaders to create: " + str(numOfLoaders)) # 10 Loaders

miniLoaders = []

for i in range(numOfLoaders):
  startIdx        = i * miniAmount
  endIdx          = (i + 1) * miniAmount if i < numOfLoaders - 1 else totalImages
  subset          = torch.utils.data.Subset(trainset, range(startIdx, endIdx))  # Creates a subset of the whole training set
  SubTrainLoader  = torch.utils.data.DataLoader(subset, batch_size=batch_size, shuffle=True, num_workers=2)

  miniLoaders.append(SubTrainLoader)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.classifier = nn.Sequential(
            nn.Linear(256 * 4 * 4, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(1024, 52),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(52, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

model         = Net()
PATH          = ('./40_epoch_32_batch_SGD_net.pth')
model.load_state_dict(torch.load(PATH))

finalLayer    = model.classifier[-1]
outerShape    = len(finalLayer.weight)
innerShape    = len(finalLayer.weight[0])
totalWeights  = outerShape * innerShape
biasCount     = len(finalLayer.bias)
paramCount    = sum(param.numel() for param in finalLayer.parameters())

criterion     = nn.CrossEntropyLoss()
nn.init.xavier_uniform(finalLayer.weight)
print("Final layer weight shape,", (finalLayer.weight).shape) # Shape that the layer requires is (10,52)
print("Final layer bias shape,", (finalLayer.bias).shape)

for param in model.parameters():
    param.requires_grad = False

populationList = finalLayer.weight.numpy()

posMinInit      = - 10
posMaxInit      = + 10
VMaxInit        = 5
VMinInit        = 0.1
dimension       = 530
interval        = 1
iterations      = 100#50*dimension
populationSize  = 100+int(dimension/10)

#variables used in SL-PSO
epsilon = dimension/100.0*0.01 # social influence of swarm centre


# function to get the mean positions of the inviduals (swarm centre)
def getcenter(pop):
    center=list()
    for j in range(dimension): # count through dimensions
        centerj = 0
        for i in pop: # for each particle
            centerj += i[j] # sum up position in dimention j
        centerj /= populationSize # Average
        center.append(centerj)
    return center


creator.create("FitnessMax", base.Fitness, weights=(1.0,)) # -1 is for minimise
creator.create("Particle", list, fitness=creator.FitnessMax, speed=list, smin=None, smax=None, best=None)
# particle rerpresented by list of 5 things
# 1. fitness of the particle,
# 2. speed of the particle which is also going to be a list,
# 3.4. limit of the speed value,
# 5. best state the particle has been in so far.

def generate(size, smin, smax):
    part = creator.Particle(random.uniform(posMinInit, posMaxInit) for _ in range(size))
    part.speed = [random.uniform(VMinInit, VMaxInit) for _ in range(size)]
    part.smin = smin #speed clamping values
    part.smax = smax
    return part



def updateParticle(part,pop,center,i):
    r1 = random.uniform(0, 1)
    r2 = random.uniform(0, 1)
    r3 = random.uniform(0, 1)

    #Randomly choose a demonstrator for particle i from any of particles 0 to i-1, the Particle i
    #updates its velocity by learning from the demonstrator and the mean position of the swarm
    demonstrator=random.choice(list(pop[0:i]))

    for j in range(dimension): # count through dimensions
        part.speed[j]=r1*part.speed[j]+r2*(demonstrator[j]-part[j])+r3*epsilon*(center[j]-part[j])
        part[j]=part[j]+part.speed[j]


def assign_weights(particle):
    particleweightsNP1 = np.array(particle)
    particleweightsNP = particleweightsNP1[:totalWeights]
    biases = np.array(particleweightsNP1[-biasCount:])
    biases = torch.from_numpy(biases).float()
    finalLayer.bias = torch.nn.Parameter(biases.float())
    reshapedWeights = particleweightsNP.reshape(outerShape,innerShape)
    torchWeights = torch.from_numpy(reshapedWeights).float()
    finalLayer.weight = torch.nn.Parameter(torchWeights.float())


# def fitness(particle):
#     assign_weights(particle)
#     correct_train = 0
#     total_train = 0
#     print("Particle")
#     for i, data in enumerate(trainloader,0):
#         model.to(device)
#         images, labels = data[0].to(device), data[1].to(device)
#         predicted=model(images)
#         loss = criterion(predicted, labels)
#         _, predictions = torch.max(predicted, 1)
#         total_train += labels.size(0)
#         correct_train += (predictions == labels).sum().item()
#         # print(f"Total train: {total_train}, Correct train: {correct_train}")

#     acc = 100*(correct_train/total_train)

#     print(f"Particle total accuracy {acc}")

#     return acc,

# def fitness(images, labels, particle):
#     assign_weights(particle)

#     model.to(device)
#     images.to(device)
#     labels.to(device)

#     predicted=model(images)

#     _, predictions = torch.max(predicted, 1)
#     total_train = labels.size(0)
#     correct_train = (predictions == labels).sum().item()
#     # print(f"Total train: {total_train}, Correct train: {correct_train}")
#     acc = 100*(correct_train/total_train)

#     print(f"Particle accuracy {acc}")

#     return acc,

def fitness(particle, miniNumber):
    assign_weights(particle)
    correct_train = 0
    total_train = 0

    for data in miniLoaders[miniNumber]:
        model.to(device)
        images, labels = data[0].to(device), data[1].to(device)
        predicted=model(images)

        _, predictions = torch.max(predicted, 1)
        total_train += labels.size(0)
        correct_train += (predictions == labels).sum().item()
        # print(f"Total train: {total_train}, Correct train: {correct_train}")
        print(f"Total train: {total_train}, Correct train: {correct_train}")
    acc = 100*(correct_train/total_train)

    print(f"Particle total accuracy {acc}")

    return acc,

toolbox = base.Toolbox()
toolbox.register("particle", generate, size=dimension, smin=-3, smax=3)
toolbox.register("population", tools.initRepeat, list, toolbox.particle)
toolbox.register("update", updateParticle)
toolbox.register("evaluate", fitness) #sphere function is built-in in DEAP

def main():
    pop = toolbox.population(n=populationSize) # Population Size
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)

    #intialize the learning probabilities
    prob=[0]*populationSize
    for i  in range(len(pop)):
        prob[populationSize - i - 1] = 1 - i/(populationSize - 1)
        prob[populationSize - i - 1] = pow(prob[populationSize - i - 1], math.log(math.sqrt(math.ceil(dimension/100.0))))

    logbook = tools.Logbook()
    logbook.header = ["gen", "evals"] + stats.fields
    miniCounter = 0

    #begin main loop
    for g in range(iterations):

        for part in pop:
            part.fitness.values = toolbox.evaluate(part, miniCounter) #actually only one fitness value

        #Sort the individuals in the swarm in ascending order. i.e., particle 0 is the best
        pop.sort(key=lambda x: x.fitness, reverse=True)
        #calculate the center (mean value) of the swarm
        center = getcenter(pop)

        for i  in reversed(range(len(pop)-1)):  # start with worst particle, and go in reverse towards best
                                                # don't do element 0 (best). Hence the i+1 below.
            if random.uniform(0, 1)<prob[i+1]: #learning probability for that particle
                toolbox.update(pop[i+1],pop,center,i+1)

        # Gather all the fitnesses in one list and print the stats
        # print every interval
        if g%interval==0: # interval
            logbook.record(gen=g, evals=len(pop), **stats.compile(pop))
            print(logbook.stream)

        miniCounter += 1

        if miniCounter >= (len(miniLoaders)):
          miniCounter = 0
          print("Mini Counter reset!")

    return pop, logbook

if __name__ == "__main__":
    pop, logbook = main()
    print(logbook)


Files already downloaded and verified
Files already downloaded and verified
Device,  cuda:0
Number of loaders to create: 50
Final layer weight shape, torch.Size([10, 52])
Final layer bias shape, torch.Size([10])


<ipython-input-1-c8b05d278d93>:110: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(finalLayer.weight)


Streaming output truncated to the last 5000 lines.
Total train: 1000, Correct train: 220
Particle total accuracy 22.0
Total train: 1000, Correct train: 224
Particle total accuracy 22.400000000000002
Total train: 1000, Correct train: 214
Particle total accuracy 21.4
Total train: 1000, Correct train: 177
Particle total accuracy 17.7
Total train: 1000, Correct train: 199
Particle total accuracy 19.900000000000002
Total train: 1000, Correct train: 250
Particle total accuracy 25.0
Total train: 1000, Correct train: 210
Particle total accuracy 21.0
Total train: 1000, Correct train: 215
Particle total accuracy 21.5
Total train: 1000, Correct train: 214
Particle total accuracy 21.4
Total train: 1000, Correct train: 181
Particle total accuracy 18.099999999999998
Total train: 1000, Correct train: 225
Particle total accuracy 22.5
Total train: 1000, Correct train: 208
Particle total accuracy 20.8
Total train: 1000, Correct train: 173
Particle total accuracy 17.299999999999997
Total train: 1000, Cor